## prisijunkime prie MongoDB

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://localhost:27017") # prisijungiame prie mongodb

In [3]:
db = client.lietuvos_imones # pasirenkame duomenu baze

In [4]:
db = client["lietuvos_imones"] # duomenu baze galima pasirinkti kaip atributa arba naudojant indexing operatoriu.

In [5]:
imones = db.imones # pasirenkame collection 

In [6]:
imones

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'lietuvos_imones'), 'imones')

## sukurkime nauja dokumenta

In [7]:
nauja_imone = {"pavadinimas":"UAB nauja imones"} # sukurkime zodyna. Zodynas bus ikeltas i db kaip naujas dokumentas

In [8]:
result = imones.insert_one(nauja_imone) # funkcijos pavadinimas sutampa su mongosh operacijos pavadinimu

In [9]:
result.inserted_id

ObjectId('626b7ae984b107332565aff8')

In [10]:
nauja_imone_1 = {"pavadinimas":"UAB nauja imone 1"} # sukuriame keleta nauju imoniu 
nauja_imone_2 = {"pavadinimas":"UAB nauja imone 2"}

In [11]:
new_result = imones.insert_many([nauja_imone_1, nauja_imone_2]) # ikeliame naujus dokumentus i db

In [12]:
new_result.inserted_ids

[ObjectId('626b7aee84b107332565aff9'), ObjectId('626b7aee84b107332565affa')]

## atraskime dokumenta

In [16]:
imones.find_one({"pavadinimas": "UAB nauja imones"})

{'_id': ObjectId('626b7ae984b107332565aff8'),
 'pavadinimas': 'UAB nauja imones'}

In [18]:
imones.find_one({"pavadinimas": 'UAB "OLAN LT"'})

{'_id': ObjectId('5fb07731261973556a11202b'),
 'pavadinimas': 'UAB "OLAN LT"',
 'kategorijos': ['Naudingosios iškasenos',
  'Akmens gaminiai',
  'Betonas ir betono gaminiai',
  'Kelių tiesimas, remontas, tiltai',
  'Statybinės medžiagos',
  'Statybos technika, įranga, nuoma'],
 'įmonės_kodas': '302583298',
 'pvm_mokėtojo_kodas': 'LT100005854919',
 'vadovas': 'Rytis Ratomskis, direktorius',
 'adresas': 'Ateities pl. 31, LT-52167 Kaunas',
 'telefonas': '+370 643 14922',
 'mobilus_telefonas': '+370 643 45605',
 'faksas': '+370 37 210757',
 'tinklalapis': 'http://www.olan.lt',
 'darbuotojai': '6 darbuotojai (apdraustieji)',
 'sd_draudėjo_kodas': '2088490',
 'vidutinis_atlyginimas': '625,89 € (2020 m. rugsėjis)',
 'vsd_įmokų_suma': '881,69 € (2020 m. rugsėjis)',
 'creditinfo_co': 'Creditinfo CO sistemoje skolų neregistruota',
 'pardavimo_pajamos': '2019: 2 000 001 - 3 000 000 € be PVM',
 'transportas': '2020-10: 4 automobiliai (priklauso)',
 'pranešimai': ['VMI: Įmonė, kuriai taikomos VMI i

In [49]:
imones_list = []
# atraskime visas imones, kurios yra  9 metu, 10 menesiu ir 1 dienos amziaus. 
for doc in imones.find({'įmonės_amžius': '9 metai 10 mėnesių 1 diena'}): 
    imones_list.append(doc) # rastas imones issaugome sarase

In [50]:
## grazinti rezultatai yra python zodyno duomenu tipo. 
## naudodamiesi python galime gauti papildoma informacija apie imones
# pvz. vidutinis atlyginimas

In [51]:
for im in imones_list:
    try:
        vid_atlyginimas = im['vidutinis_atlyginimas'].split('€')[0].strip().replace(',', '.').replace(' ', '')
        vid_atlyginimas = float(vid_atlyginimas)
        print(im['pavadinimas'], vid_atlyginimas)
    except KeyError:
        vid_atlyginimas = 'neapibreztas'
    

UAB "OLAN LT" 625.89
Senjorų Vila, VšĮ 1173.07
Baltmed, UAB 1578.51
Dzūko Medis, UAB 940.7
Partnervetas, UAB 1949.74
UAB Vaidos klinika 1527.85
KIETAVIŠKIŲ AUTOCENTRAS, UAB 796.0


## pasinaudokime tekstine paieska

In [54]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [56]:
search_text = 'maxima'
for im in imones.find({"$text": {"$search": search_text}}).limit(2):
    pp.pprint(im)

{   '_id': ObjectId('5fbe877e025fc0398f7f8638'),
    'adresas': 'Naugarduko g. 84, LT-03160 Vilnius',
    'creditinfo_co': 'Creditinfo CO sistemoje skolų neregistruota',
    'darbuotojai': '1 darbuotojas (apdraustasis)',
    'meta': {   'key': 'uab_maxima',
                'origin': 'req-data-scraper-v2',
                'timestamp': '2020-11-25T16:34:06.894240+00:00',
                'version': 3},
    'pardavimo_pajamos': '2015: 0 - 5 000 € be PVM Pajamų istorija »',
    'pavadinimas': 'Maxima, UAB',
    'sd_draudėjo_kodas': '448311',
    'telefonas': '+370 5 2744337',
    'vadovas': 'Vilius Rimkus, direktorius',
    'įmonės_amžius': '29 metai 9 mėnesiai 18 dienų',
    'įmonės_kodas': '132477256',
    'įvertinimas': '5.5 / 10 (įvertino 11)'}
{   '_id': ObjectId('5fbe27eb1a11a488fa67e3c3'),
    'adresas': 'Kaunas',
    'creditinfo_co': 'Creditinfo CO sistemoje skolų neregistruota',
    'meta': {   'key': 'maxima_pecunia',
                'origin': 'req-data-scraper-v2',
              

# uzduotis

In [57]:
# 1. atraskite imoniu skaiciu, kurios turi daugiau nei 10 darbuotoju
# 2. atraskite imoniu skaiciu, kurios yra registruotos Klaipedoje

# hint - galite naudoti regexp
# https://www.mongodb.com/docs/manual/reference/operator/query/regex/


In [59]:
# PVZ.
# imones, kuriu vadovo vardas yra Vilius
viliaus_imones = [*imones.find({'vadovas':{'$regex':'^Vilius'}})]

In [60]:
len(viliaus_imones) # tokiu imoniu yra 634

634

In [61]:
viliaus_imones[0]

{'_id': ObjectId('5fb07755261973556a112075'),
 'pavadinimas': 'UAB LANEKA',
 'kategorijos': ['Apželdinimas ir aplinkos tvarkymas', 'Akmens gaminiai'],
 'įmonės_kodas': '302762237',
 'pvm_mokėtojo_kodas': 'LT100006837117',
 'vadovas': 'Vilius Perkauskas, direktorius',
 'adresas': 'Statybininkų g. 15, LT-98129 Skuodas',
 'mobilus_telefonas': '+370 679 26006',
 'tinklalapis': 'http://zemaitijosgerbuvis.lt',
 'facebook_paskyra': 'https://www.facebook.com/ZemaitijosGerbuvis/',
 'darbuotojai': '5 darbuotojai (apdraustieji)',
 'sd_draudėjo_kodas': '2152927',
 'creditinfo_co': 'Creditinfo CO sistemoje skolų neregistruota',
 'pardavimo_pajamos': '2019: 50 001 - 100 000 € be PVM',
 'transportas': '2020-10: 4 automobiliai (priklauso)',
 'pranešimai': ['VMI: Įmonė, kuriai taikomos VMI ir SODRA mokestinės pagalbos priemonės dėl COVID-19.'],
 'įmonės_amžius': '8 metai 7 mėnesiai 3 dienos',
 'įvertinimas': '6.3 / 10 (įvertino 31)',
 'prekinis_ženklas': 'Žemaitijos Gerbūvis',
 'šūkis': '"Jūsų svajonių